# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


In [0]:
tf.__version__

'2.1.0'

In [0]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 13s 0us/step


In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [0]:
y_train.reshape(-1,1)
y_test.reshape(-1,1)

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]], dtype=uint8)

In [0]:
p = y_train>=5

p = p.reshape(50000,)
p.shape

(50000,)

In [0]:
trainX59 = x_train[p,:,:,:] 
trainX59.shape

(25000, 32, 32, 3)

In [0]:
trainY59 = y_train[p,:]
trainY59.shape

(25000, 1)

In [0]:
p = y_test>=5

p = p.reshape(10000,)
p.shape


(10000,)

In [0]:
testY59 = y_test[p,:]
testY59.shape

(5000, 1)

In [0]:
testX59 = x_test[p,:,:,:] 
testX59.shape

(5000, 32, 32, 3)

In [0]:
p = y_train<5

p = p.reshape(50000,)
p.shape

(50000,)

In [0]:
trainX04 = x_train[p,:,:,:] 
trainX04.shape

(25000, 32, 32, 3)

In [0]:
trainY04 = y_train[p,:]
trainY04.shape

(25000, 1)

In [0]:
p = y_test<5

p = p.reshape(10000,)
p.shape

(10000,)

In [0]:
testX04 = x_test[p,:,:,:] 
testX04.shape

(5000, 32, 32, 3)

In [0]:
testY04 = y_test[p,:]
testY04.shape

(5000, 1)

In [0]:
print(np.unique(trainY59))
print(np.unique(testY59))
print(np.unique(trainY04))
print(np.unique(testY04))

[5 6 7 8 9]
[5 6 7 8 9]
[0 1 2 3 4]
[0 1 2 3 4]


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
train_y04 = tf.keras.utils.to_categorical(trainY04, num_classes=5)
test_y04 = tf.keras.utils.to_categorical(testY04, num_classes=5)
train_y04.shape

(25000, 5)

In [0]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder()
train_y59 = oh.fit_transform(trainY59)
test_y59 = oh.transform(testY59)
train_y59.shape

(25000, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,vertical_flip=True,zoom_range=0.3)

In [0]:
#Clear any existing model in memory
tf.keras.backend.clear_session()

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add first convolutional layer    
model.add(tf.keras.layers.Conv2D(128,kernel_size= (3, 3), activation = 'relu',input_shape=trainX04.shape[1:]))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Second convolutional layer    
model.add(tf.keras.layers.Conv2D(128,kernel_size=(3, 3), activation = 'relu'))

#Add maxpool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add a dropout layer
model.add(tf.keras.layers.Dropout(0.2))

In [0]:
#Add third convolutional layer
model.add(tf.keras.layers.Conv2D(256,kernel_size= (3, 3),activation='relu'))

#Add maxpool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add fourth convolutional layer
model.add(tf.keras.layers.Conv2D(256,kernel_size= (3,3),activation='relu'))

#Add max pool layer
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

In [0]:
#Flatten out the data
model.add(tf.keras.layers.Flatten())

# Adding a dense layer
model.add(tf.keras.layers.Dense(128,activation = 'relu'))

#Adding a drouput layer
model.add(tf.keras.layers.Dropout(0.2))

#Adding output layer
model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [0]:
sgd = tf.keras.optimizers.SGD(lr = 0.1, decay=1e-6, momentum=0.7,nesterov=True)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0

In [0]:
model.fit(trainX04,train_y04,
                    epochs=5,
                        validation_data=(testX04,test_y04))

Train on 25000 samples, validate on 5000 samples
Epoch 1/5
25000/25000 [==============================] - 17s 675us/sample - loss: 1.0291 - accuracy: 0.5899 - val_loss: 1.3550 - val_accuracy: 0.5384
Epoch 2/5
25000/25000 [==============================] - 9s 352us/sample - loss: 0.7473 - accuracy: 0.7177 - val_loss: 0.7491 - val_accuracy: 0.7042
Epoch 3/5
25000/25000 [==============================] - 9s 355us/sample - loss: 0.6101 - accuracy: 0.7740 - val_loss: 0.9600 - val_accuracy: 0.6518
Epoch 4/5
25000/25000 [==============================] - 9s 352us/sample - loss: 0.5203 - accuracy: 0.8096 - val_loss: 0.5340 - val_accuracy: 0.8058
Epoch 5/5
25000/25000 [==============================] - 9s 358us/sample - loss: 0.4384 - accuracy: 0.8437 - val_loss: 0.7344 - val_accuracy: 0.7162


In [0]:
model.output

<tf.Tensor 'dense_1/Identity:0' shape=(None, 5) dtype=float32>

In [0]:
new_model=model

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
   
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in new_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5b6f9bfd0>,conv2d,True
1,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fe5b6f9b6d8>,batch_normalization,True
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5b67456d8>,conv2d_1,True
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5b676df60>,max_pooling2d,True
4,<tensorflow.python.keras.layers.core.Dropout object at 0x7fe5a05458d0>,dropout,True
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5b6745da0>,conv2d_2,True
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5b6745c88>,max_pooling2d_1,True
7,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fe5a05535f8>,batch_normalization_1,True
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5a05637f0>,conv2d_3,True
9,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5a04ef7f0>,max_pooling2d_2,True


In [0]:
set_trainable = False
for layer in new_model.layers[:11]:
    layer.trainable = False

layers = [(layer, layer.name, layer.trainable) for layer in new_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5b6f9bfd0>,conv2d,False
1,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fe5b6f9b6d8>,batch_normalization,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5b67456d8>,conv2d_1,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5b676df60>,max_pooling2d,False
4,<tensorflow.python.keras.layers.core.Dropout object at 0x7fe5a05458d0>,dropout,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5b6745da0>,conv2d_2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5b6745c88>,max_pooling2d_1,False
7,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fe5a05535f8>,batch_normalization_1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5a05637f0>,conv2d_3,False
9,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5a04ef7f0>,max_pooling2d_2,False


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
#new_model=model
new_model.pop()

In [0]:
#Check if layers frozen
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0

In [0]:
new_model.add(tf.keras.layers.Dense(128,activation='relu'))
#Add Dropout
new_model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
new_model.add(tf.keras.layers.Dense(5,activation='softmax'))


In [0]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0

In [0]:
new_model.fit(trainX59,train_y59.todense(),epochs=5, shuffle=True,  validation_data=(testX59,test_y59.todense()), verbose=2)

Train on 25000 samples, validate on 5000 samples
Epoch 1/5
25000/25000 - 5s - loss: 0.8486 - accuracy: 0.6888 - val_loss: 0.5580 - val_accuracy: 0.7972
Epoch 2/5
25000/25000 - 4s - loss: 0.5719 - accuracy: 0.7957 - val_loss: 0.4774 - val_accuracy: 0.8270
Epoch 3/5
25000/25000 - 4s - loss: 0.5005 - accuracy: 0.8164 - val_loss: 0.4638 - val_accuracy: 0.8330
Epoch 4/5
25000/25000 - 5s - loss: 0.4639 - accuracy: 0.8326 - val_loss: 0.4749 - val_accuracy: 0.8280
Epoch 5/5
25000/25000 - 4s - loss: 0.4423 - accuracy: 0.8382 - val_loss: 0.4396 - val_accuracy: 0.8394


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [0]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [0]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.001, max_df=0.20)
vectors = vectorizer.fit_transform(twenty_train.data)
vectors_test = vectorizer.transform(twenty_test.data)
x1 = vectors
y1 = twenty_train.target
x2 = vectors_test
y2 = twenty_test.target
x = np.array(np.r_[x1.todense(), x2.todense()])
y = np.r_[y1, y2]

In [0]:
pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

,00,000,0001,0010580b,01,0100,01580,02,0200,02118,02173,023044,024103,03,0349,0358,04,040,041343,05,0511,06,0600,0608,06320,07,08,081052,084042,09,095220,0x100,10,100,1000,101,10101,101010,1013,102,...,york,young,younger,youngsters,youngstown,yourselfers,youself,youth,yoyo,yozzo,yr,yrs,ysu,yugoslavia,yuv,z1,z5,za,zabriskie,zaphod,zazen,zcomm,zealand,zen,zenkar,zero,zeus,zhao,zhenghao,zillions,zinc,zion,zip,zisfein,zmodem,zoerasterism,zola,zoom,zooming,zyeh
0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.084371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.06375,0.0,0.0,0.0,0.065943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2253,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.066048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2254,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2255,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.066238,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
# use logistic regression with text column only
logreg = LogisticRegression(C=1e9)
logreg.fit(vectors, y1)
y_pred_class = logreg.predict(vectors_test)
print(metrics.accuracy_score(y2, y_pred_class))

0.9221038615179761
